In [1]:
import nltk
nltk.download('abc')
import nltk
from nltk.corpus import abc
word_list = (nltk.corpus.abc.words())
sentence_list = nltk.corpus.abc.sents()

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Package abc is already up-to-date!


In [0]:
import numpy as np
###Change the vocab size my removing the stopwords 
vocab_size = len(word_list)
window_size = 5
vector_dim = 300

"""For similarity checking"""
###Change a bit
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)




In [0]:
def tokenizer(corpus):
  """Corpus already is changed to tokens we just need to change them to lower"""
  final_list = []
  for i in corpus:
    ###Need to remove the stopwords
    final_list.append(i.lower())
  return(final_list)

def id_mapping(token):
  word_to_id_mapping = {}
  id_to_word_mapping = {}
  ###I think mapping is a bit faulty cause the id changes when the word is repeated
  i=0;
  for text in token:
    #if(i==0):
      #print(text)
    word_to_id_mapping[text] = i
    id_to_word_mapping[i] = text
    i=i+1

  return word_to_id_mapping,id_to_word_mapping

token = tokenizer(word_list)
word_to_id_mapping , id_to_word_mapping = id_mapping(token)

data = []
for word in word_to_id_mapping:
  data.append(word_to_id_mapping[word])


In [4]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, merge
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
import tensorflow as tf

sample_matrix = sequence.make_sampling_table(vocab_size) 
couple,label = skipgrams(data, vocab_size, window_size=window_size, sampling_table =sample_matrix)

"""Couple is the skipgram 
Label is the one hot encoding"""


Using TensorFlow backend.


'Couple is the skipgram \nLabel is the one hot encoding'

In [0]:
"""Embedding"""

target_input = Input((1,))
context_input = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(target_input)
context = embedding(context_input)

target = Reshape((vector_dim,1))(target) #Reshaping according to vector dimension size
context = Reshape((vector_dim,1))(context)

In [6]:
"""Similarity Function"""
from keras.layers import Input, Dense, Reshape, merge, Dot


similar = Dot(axes=1,normalize = True)([target, context])
dot_product = Dot(axes=1, normalize = False)([target, context])
dot_product = Reshape((1,))(dot_product)

output = Dense(1, activation='sigmoid')(dot_product)

model = Model(input=[target_input, context_input], output=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


In [7]:
validation_model = Model(input=[target_input, context_input], output=similar)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("do...)`
  """Entry point for launching an IPython kernel.


In [0]:
## Can be chaanged
word_target, word_context = zip(*couple) 
word_target = np.array(word_target)
word_context = np.array(word_context)

In [9]:
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.cm as cm

for cnt in range(200):
  loss = model.train_on_batch([word_target, word_context], label)
  print("Iteration {}, loss={}".format(cnt, loss))
  
output1 = open('/content/drive/My Drive/Assignment3/model_full'+ '.pickle', 'wb')
pickle.dump(model, output1)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Iteration 0, loss=[0.69317275, 0.49945477]
Iteration 1, loss=[0.69308156, 0.5046868]
Iteration 2, loss=[0.69295585, 0.5119516]
Iteration 3, loss=[0.6928043, 0.5206319]
Iteration 4, loss=[0.69263107, 0.5307095]
Iteration 5, loss=[0.69243884, 0.54145515]
Iteration 6, loss=[0.69222945, 0.5535873]
Iteration 7, loss=[0.692004, 0.5664867]
Iteration 8, loss=[0.6917635, 0.5799657]
Iteration 9, loss=[0.6915086, 0.5942643]
Iteration 10, loss=[0.6912396, 0.6092959]
Iteration 11, loss=[0.6909571, 0.6248547]
Iteration 12, loss=[0.6906613, 0.6408016]
Iteration 13, loss=[0.6903524, 0.6570681]
Iteration 14, loss=[0.69003034, 0.6736234]
Iteration 15, loss=[0.68969536, 0.69029427]
Iteration 16, loss=[0.6893473, 0.7072053]
Iteration 17, loss=[0.6889862, 0.7241596]
Iteration 18, loss=[0.68861187, 0.74139553]
Iteration 19, loss=[0.6882242, 0.75844556]
Iteration 20, loss=[0.68782294, 0.7756183]
Iteration 21, loss=[0.6874079, 0.7922116]
Iteration 22, loss=[0.6869789, 0.8086279]
Iteration 23, loss=[0.6865355,